In [1]:
import pandas as pd

In [ ]:
# df_train = pd.read_csv('samsum-train.csv').drop(columns=["id"]).map(str)
# df_test = pd.read_csv('samsum-test.csv').drop(columns=["id"]).map(str)
# df_validate = pd.read_csv('samsum-validation.csv').drop(columns=["id"]).map(str)
# data_test = pd.read_csv('samsum-test.csv').drop(columns=["id"]).map(str)

df_train = pd.read_csv('/kaggle/input/samsum-dataset-text-summarization/samsum-train.csv').drop(columns=["id"]).map(str)
df_test = pd.read_csv('/kaggle/input/samsum-dataset-text-summarization/samsum-test.csv').drop(columns=["id"]).map(str)
df_validate = pd.read_csv('/kaggle/input/samsum-dataset-text-summarization/samsum-validation.csv').drop(columns=["id"]).map(str)
data_test = pd.read_csv('/kaggle/input/samsum-dataset-text-summarization/samsum-test.csv').drop(columns=["id"]).map(str)

In [3]:
start_token = '<|sos|>'
end_token = '<|eos|>'
pad_token = '<|pad|>'

In [ ]:
df_train['summary'] = df_train['summary'].apply(lambda x: start_token + x + end_token)
df_test['summary'] = df_test['summary'].apply(lambda x: start_token + x + end_token)
df_validate['summary'] = df_validate['summary'].apply(lambda x: start_token + x + end_token)

In [5]:
from transformers import AutoTokenizer

In [6]:
tokenizer = AutoTokenizer.from_pretrained('gpt2')

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [7]:
len(tokenizer)

50257

In [8]:
tokenizer.add_special_tokens({
    "pad_token": pad_token,
    "additional_special_tokens": [start_token, end_token]
})

3

In [9]:
len(tokenizer)

50260

In [10]:
MAX_LENGTH = 512

In [11]:
decoder_length = 128

In [12]:
inputs = tokenizer(
    df_train['dialogue'].tolist(), 
    max_length=MAX_LENGTH, 
    padding="max_length", 
    truncation=True, 
    return_tensors="pt"
)

targets = tokenizer(
    df_train['summary'].tolist(), 
    max_length=decoder_length, 
    padding="max_length", 
    truncation=True, 
    return_tensors="pt"
)

In [13]:
df_train.shape

(14732, 2)

In [14]:
import tensorflow as tf

In [15]:
inputs

{'input_ids': tensor([[ 5840,  5282,    25,  ..., 50257, 50257, 50257],
        [   46, 16017,   544,  ..., 50257, 50257, 50257],
        [14967,    25, 15902,  ..., 50257, 50257, 50257],
        ...,
        [ 7554,    25,  3887,  ..., 50257, 50257, 50257],
        [43187,    25, 23420,  ..., 50257, 50257, 50257],
        [41072,    25,   389,  ..., 50257, 50257, 50257]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [16]:
targets

{'input_ids': tensor([[50258,  5840,  5282,  ..., 50257, 50257, 50257],
        [50258,    46, 16017,  ..., 50257, 50257, 50257],
        [50258, 26374,   743,  ..., 50257, 50257, 50257],
        ...,
        [50258, 16504,   318,  ..., 50257, 50257, 50257],
        [50258,    34, 25418,  ..., 50257, 50257, 50257],
        [50258, 41072,   290,  ..., 50257, 50257, 50257]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

In [17]:
BUFFER_SIZE = 20000
BATCH_SIZE = 16

In [18]:
inputs_ids = inputs['input_ids']
targets_ids = targets['input_ids']

In [19]:
inputs_tensor = tf.cast(inputs_ids, dtype=tf.int32)
targets_tensor = tf.cast(targets_ids, dtype=tf.int32)

In [20]:
ds = tf.data.Dataset.from_tensor_slices((inputs_tensor, targets_tensor)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [21]:
from tensorflow.keras.layers import Dense, Layer

In [22]:
class MultiHeadAttention(Layer):
    def __init__(self, model_dim, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.model_dim = model_dim
        self.num_heads = num_heads

        self.head_dim = model_dim // self.num_heads

        self.wq = Dense(model_dim)
        self.wk = Dense(model_dim)
        self.wv = Dense(model_dim)

        self.linear_dense = Dense(model_dim)

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.head_dim))
        return tf.transpose(x, perm=[0, 2, 1, 3])
    
    def compute_scaled_dot_product_attention(self, query, key, value, mask):
        dot_product = tf.matmul(query, key, transpose_b=True)
        
        key_dim = tf.cast(tf.shape(key)[-1], tf.float32)
        scaled_dot_product = dot_product / tf.math.sqrt(key_dim)
        
        if mask is not None:
            scaled_dot_product += (mask * -1e9)
        
        attention_weights = tf.nn.softmax(scaled_dot_product, axis=-1)
        
        output_values = tf.matmul(attention_weights, value)
        
        return output_values, attention_weights

    def call(self, q, k, v, mask):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)
        k = self.wk(k)
        v = self.wv(v)

        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)

        values, attention_weights = self.compute_scaled_dot_product_attention(q, k, v, mask)

        values = tf.transpose(values, perm=[0, 2, 1, 3])

        concat_values = tf.reshape(values, (batch_size, -1, self.model_dim))

        output = self.linear_dense(concat_values)

        return output, attention_weights

In [23]:
from tensorflow.keras.layers import Dropout

In [24]:
class PointwiseFeedForward(Layer):
    def __init__(self, model_dim, hidden, drop_prob=0.1):
        super(PointwiseFeedForward, self).__init__()
        self.linear1 = Dense(hidden, activation='relu')
        self.linear2 = Dense(model_dim)
        self.dropout = Dropout(rate=drop_prob)

    def call(self, x):
        x = self.linear1(x)
        x = self.dropout(x)
        x = self.linear2(x)
        return x

In [25]:
from tensorflow.keras.layers import LayerNormalization

In [26]:
class EncoderLayer(Layer):
    def __init__(self, model_dim, num_heads, hidden, rate=0.1):
        super(EncoderLayer, self).__init__()

        self.attention = MultiHeadAttention(model_dim, num_heads)
        self.norm1 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate=rate)

        self.ffn = PointwiseFeedForward(model_dim, hidden, drop_prob=rate)
        self.norm2 = LayerNormalization(epsilon=1e-6)
        self.dropout2 = Dropout(rate=rate)

    def call(self, x, training, mask):
        residual_x = tf.identity(x)
        x, _ = self.attention(x, x, x, mask)
        x = self.dropout1(x, training=training)
        x = self.norm1(residual_x + x)
        residual_x = tf.identity(x)

        x = self.ffn(x)
        x = self.dropout2(x, training=training)
        x = self.norm2(residual_x + x)

        return x

In [27]:
class DecoderLayer(Layer):
    def __init__(self, model_dim, num_heads, hidden, rate=0.1):
        super(DecoderLayer, self).__init__()

        self.attention1 = MultiHeadAttention(model_dim, num_heads)
        self.attention2 = MultiHeadAttention(model_dim, num_heads)

        self.ffn = PointwiseFeedForward(model_dim, hidden, drop_prob=rate)

        self.norm1 = LayerNormalization(epsilon=1e-6)
        self.norm2 = LayerNormalization(epsilon=1e-6)
        self.norm3 = LayerNormalization(epsilon=1e-6)

        self.dropout1 = Dropout(rate=rate)
        self.dropout2 = Dropout(rate=rate)
        self.dropout3 = Dropout(rate=rate)

    def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
        output1, attention_weights1 = self.attention1(x, x, x, look_ahead_mask)
        output1 = self.dropout1(output1, training=training)
        output1 = self.norm1(output1 + x)

        output2, attention_weights2 = self.attention2(output1, enc_output, enc_output,  padding_mask)
        output2 = self.dropout2(output2, training=training)
        output2 = self.norm2(output2 + output1)

        output3 = self.ffn(output2)
        output3 = self.dropout3(output3, training=training)
        output3 = self.norm3(output3 + output2)

        return output3, attention_weights1, attention_weights2

In [28]:
from tensorflow.keras.layers import Embedding

In [29]:
import numpy as np

In [30]:
def generate_positional_encoding(max_position, model_dim):
    positions = np.arange(max_position).reshape(-1, 1)
    dimensions = np.arange(model_dim).reshape(1, -1)
    
    angle_exponents = (2 * (dimensions // 2)) / np.float32(model_dim)
    angle_rates = 1 / np.power(10000, angle_exponents)
    
    angle_values = positions * angle_rates
    angle_values[:, ::2] = np.sin(angle_values[:, 0::2])
    angle_values[:, ::2] = np.cos(angle_values[:, 1::2])
    
    positional_encoding = np.expand_dims(angle_values, axis=0)
    positional_encoding = tf.convert_to_tensor(positional_encoding, dtype=tf.float32)

    return positional_encoding

In [31]:
class Encoder(Layer):
    def __init__(self,  model_dim, num_layers, num_heads, hidden, input_vocab_size, max_pos_encoding, rate=0.1):
        super(Encoder, self).__init__()

        self.model_dim = model_dim
        self.num_layers = num_layers

        self.embedding = Embedding(input_vocab_size, model_dim)
        self.pos_encoding = generate_positional_encoding(max_pos_encoding, self.model_dim)

        self.enc_layers = [EncoderLayer(model_dim, num_heads, hidden, rate) for _ in range(num_layers)]

    def call(self, x, training, mask):
        seq_len = tf.shape(x)[1]

        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.model_dim, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training=training, mask=mask)

        return x

In [32]:
class Decoder(Layer):
    def __init__(self, model_dim, num_layers,  num_heads, hidden, target_vocab_size, max_pos_encoding, rate=0.1):
        super(Decoder, self).__init__()

        self.model_dim = model_dim
        self.num_layers = num_layers

        self.embedding = Embedding(target_vocab_size, model_dim)
        self.pos_encoding = generate_positional_encoding(max_pos_encoding, model_dim)

        self.dec_layers = [DecoderLayer(model_dim, num_heads, hidden, rate) for _ in range(num_layers)]
        self.dropout = Dropout(rate)

    def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
        seq_len = tf.shape(x)[1]
        attention_weights = {}

        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.model_dim, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x, block1, block2 = self.dec_layers[i](x, enc_output, training=training, look_ahead_mask=look_ahead_mask, padding_mask=padding_mask)
            attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
            attention_weights['decoder_layer{}_block2'.format(i+1)] = block2

        return x, attention_weights

In [33]:
from tensorflow.keras.models import Model

In [34]:
class Transformer(Model):
    def __init__(self, model_dim, num_layers, num_heads, hidden, input_vocab_size, target_vocab_size, max_pos_input, max_pos_target, rate=0.1):
        super(Transformer, self).__init__()

        self.encoder = Encoder(model_dim, num_layers, num_heads, hidden, input_vocab_size, max_pos_input, rate)
        self.decoder = Decoder(model_dim, num_layers, num_heads, hidden, target_vocab_size, max_pos_target, rate)
        self.final_layer = tf.keras.layers.Dense(target_vocab_size)

    def call(self, inp, tar, training, enc_padding_mask, look_ahead_mask, dec_padding_mask):
        enc_output = self.encoder(inp, training=training, mask=enc_padding_mask)
        dec_output, attention_weights = self.decoder(tar, enc_output, training=training, look_ahead_mask=look_ahead_mask, padding_mask=dec_padding_mask)
        final_output = self.final_layer(dec_output)
        return final_output, attention_weights

In [35]:
from tensorflow.keras.optimizers.schedules import LearningRateSchedule

In [36]:
class CustomSchedule(LearningRateSchedule): 
    def __init__(self, model_dim, warmup_steps=4000):
        super().__init__()

        self.model_dim = model_dim
        self.model_dim = tf.cast(self.model_dim, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, dtype=tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step* (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.model_dim) * tf.math.minimum(arg1, arg2)

In [37]:
model_dim = 128

In [38]:
learning_rate = CustomSchedule(model_dim)
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

In [39]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

In [40]:
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

In [41]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
# val_loss = tf.keras.metrics.Mean(name='val_loss')

In [42]:
tokenizer.vocab_size

50257

In [43]:
num_layers = 4
hidden = 512
num_heads = 8

In [44]:
transformer = Transformer(
    model_dim,
    num_layers,
    num_heads,
    hidden,
    tokenizer.vocab_size,
    tokenizer.vocab_size,
    max_pos_input=MAX_LENGTH,
    max_pos_target=decoder_length,
)

In [45]:
def create_look_ahead_mask(size):
    ones = tf.ones((size, size))
    req_matrix = tf.linalg.band_part(ones, -1, 0)
    toggle_req_matrix = 1 - req_matrix
    mask = toggle_req_matrix
    return mask

def create_padding_mask(seq):
    padding_mask = tf.math.equal(seq, 0)
    padding_mask = tf.cast(padding_mask, tf.float32)
    padding_mask = tf.expand_dims(padding_mask, axis=1)
    padding_mask = tf.expand_dims(padding_mask, axis=2)
    return padding_mask

def create_masks(inputs, targets):
    enc_padding_mask = create_padding_mask(inputs)
    dec_padding_mask = create_padding_mask(inputs)
    look_ahead_mask = create_look_ahead_mask(tf.shape(targets)[1])
    dec_target_padding_mask = create_padding_mask(targets)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)
    return enc_padding_mask, combined_mask, dec_padding_mask

In [46]:
@tf.function
def train_step(inp, tar):
    tar_inp = tar[:, :-1]
    tar_real = tar[:, 1:]

    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)

    with tf.GradientTape() as tape:
        predictions, _ = transformer(
            inp, 
            tar_inp, 
            training=True,  
            enc_padding_mask=enc_padding_mask, 
            look_ahead_mask=combined_mask, 
            dec_padding_mask=dec_padding_mask
        )
        loss = loss_function(tar_real, predictions)

    gradients = tape.gradient(loss, transformer.trainable_variables)
    optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

    train_loss(loss)

In [47]:
# def validate_step(inp, tar):
#     tar_inp = tar[:, :-1]
#     tar_real = tar[:, 1:]

#     enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)

#     predictions, _ = transformer(
#         inp, 
#         tar_inp, 
#         training=False,  # Set training to False for validation
#         enc_padding_mask=enc_padding_mask, 
#         look_ahead_mask=combined_mask, 
#         dec_padding_mask=dec_padding_mask
#     )
#     loss = loss_function(tar_real, predictions)
#     return loss

In [48]:
import time

In [ ]:
EPOCHS = 5
for epoch in range(EPOCHS):
    start = time.time()

    train_loss.reset_state()
    # val_loss.reset_state()

    # Training
    for (batch, (inp, tar)) in enumerate(ds):
        train_step(inp, tar)

    # Validation
    # for (batch, (inp, tar)) in enumerate(val_ds):
    #     val_loss(validate_step(inp, tar))

    print('Epoch {} Train Loss {:.4f} '.format(epoch + 1, train_loss.result()))
    # print('Epoch {} Train Loss {:.4f} Validation Loss {:.4f}'.format(epoch + 1, train_loss.result(), val_loss.result()))
    print('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

Epoch 1 Train Loss 4.233
Time taken for 1 epoch: 300.13231992721558  secs

Epoch 2 Train Loss 2.3458
Time taken for 1 epoch: 265.2878189086914 secs

Epoch 3 Train Loss 2.1968
Time taken for 1 epoch: 267.27801132202148 secs

Epoch 4 Train Loss 1.84
Time taken for 1 epoch: 263.36598732524287 secs

Epoch 5 Train Loss 1.67
Time taken for 1 epoch: 265.48732954987592 secs


In [ ]:
def evaluate(dialogue):
    input_ids = tokenizer.encode(
        dialogue, 
        return_tensors="tf", 
        max_length=MAX_LENGTH, 
        padding='max_length', 
        truncation=True
    )

    encoder_input = input_ids 

    start_token_id = tokenizer.convert_tokens_to_ids(start_token)
    end_token_id = tokenizer.convert_tokens_to_ids(end_token)

    output = tf.expand_dims([start_token_id], 0)  

    for _ in range(decoder_length):
        enc_padding_mask, combined_mask, dec_padding_mask = create_masks(encoder_input, output)

        predictions, _ = transformer(
            encoder_input,
            output,
            training=False,
            enc_padding_mask=enc_padding_mask,
            look_ahead_mask=combined_mask,
            dec_padding_mask=dec_padding_mask
        )

        predictions = predictions[:, -1:, :]  

        predicted_id = tf.argmax(predictions, axis=-1, output_type=tf.int32)  

        if predicted_id[0][0].numpy() == end_token_id:
            break

        output = tf.concat([output, predicted_id], axis=-1)  

    generated_ids = output.numpy()[0][1:]  
    return generated_ids

In [ ]:
def summarize(dialogue):
    generated_ids = evaluate(dialogue)
    summary = tokenizer.decode(
        generated_ids, 
        skip_special_tokens=True, 
        clean_up_tokenization_spaces=True
    )
    return summary.strip()

In [ ]:
for i in range(50, 60):
    dialogue = df_test.dialogue.iloc[i]
    print("\nSample Text:")
    print(dialogue)

    print("\nPredicted Summary:")
    print(summarize(dialogue))

    print("\nReference Summary:")
    print(data_test.summary.iloc[i])


Sample Text:
Nick: You look absolutely gorgeous and have a lovely smile. 
Nick: Would love to get to know you a bit more. How about we meet up for a drink sometime?
Jane: Hmmm... You're shooting a bit above your range aren't you?
Nick: Why would you think that hon?
Jane: Because I'm not that desperate.
Nick: That was a bit below the belt.
Nick: You're nice but you're not THAT hot.
Jane: Oh is your poor little dick shriveling at the thought?
Nick: Actually I'll take it back. Forget about the drink.
Nick: Forget I ever wrote to you.
Jane: Bye loser!
Nick: Fucking bitch!
Jane: You're welcome!

Predicted Summary:
sam is angry with a video that he doesn't want to watch a video video with him

Reference Summary: Nick finds Jane pretty and invites her for a drink to get to know her better. Jane rejects Nick and is unpleasant to him. Nick suggests Jane to forget about their conversation.

Sample Text:
Adam: My friend told me he saw Tim with a guy.
Nate: And?
Adam: <file_photo>
Nate: omg
Julia

In [ ]:
from rouge_score import rouge_scorer, scoring

In [ ]:
def evaluate_summaries(data, num_samples=100):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    aggregator = scoring.BootstrapAggregator()

    for i in range(num_samples):
        dialogue = data['dialogue'].iloc[i]
        reference_summary = data['summary'].iloc[i]
        generated_summary = summarize(dialogue)

        scores = scorer.score(reference_summary, generated_summary)
        aggregator.add_scores(scores)

    result = aggregator.aggregate()
    print("ROUGE-1: {:.4f}".format(result['rouge1'].mid.fmeasure))
    print("ROUGE-2: {:.4f}".format(result['rouge2'].mid.fmeasure))
    print("ROUGE-L: {:.4f}".format(result['rougeL'].mid.fmeasure))

ROUGE-1: 0.2452
ROUGE-2: 0.0456
ROUGE-L: 0.1118


In [ ]:
evaluate_summaries(df_test, num_samples=100)